In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm
tqdm.pandas()

# Convert MTR Data from txt to DataFrame

In [77]:
MTR_df = pd.DataFrame(columns=['CODE','fyear','MTR_BI','MTR_AI','Permno','gvkey'])
with open('Generated Data/mtr8021.txt') as f:
    lines = f.readlines()

In [4]:
for line in tqdm(lines):
    row = line.split(' ')
    MTR_df = MTR_df.append({'CODE': row[0],
                   'fyear': row[1],
                   'MTR_BI': row[2],
                   'MTR_AI': row[3],
                   'Permno': row[4],
                   'gvkey': row[5]}, ignore_index= True)

100%|███████████████████████████████████| 247267/247267 [48:02<00:00, 85.78it/s]


In [5]:
MTR_df.to_csv('MTR.csv', index=False)

In [6]:
MTR_df.head()

,CODE,fyear,MTR_BI,MTR_AI,Permno,gvkey
0,00002110,1987,0.39,0.39,.,-999\n
1,00002110,1989,0.34,0.39,.,-999\n
2,00002110,1990,0.39,0.39,.,-999\n
3,00002110,1991,0.208948,0,.,-999\n
4,00002110,1992,0.02522,0,.,-999\n


# MTR Data from Duke
reg years (2009-2021) not fully present per firm, each firm must have 13 years slots filled with missing if not given \
some missing MTR data in '.' instead of None/NaN

The first column has 8 characters. \
**The first 6 characters are the company's CNUM from COMPUSTAT.** 
**The seventh through eighth characters are the first two digits from the company's CIC code from Compustat.** \
The eight characters are analogous to a CUSIP (first 8 digits). \
The original idea behind using eight characters was that you could match exactly to each firm's 8-digit CUSIP. \
Some firms, like GM, have several Compustat 6-digit CNUMs (one for parent firm, one each for various subsidiaries), so to match precisely, you need to match on more than six digits. \
Unfortunately, Compustat changes CIC codes so often, that using the 8-digit code in my file from, say, 1998 will not match perfectly for 1997, 1996, etc. for some firms. In other words, you may be stuck matching by 6-digit CNUM and hand-checking for possible problems for firms that have subsidiaries or tracking stock. Note: this is not a problem for most firms. Also, the 6-digit/8-digit difference is a potential issue for using ANY Compustat data and is not specific to the tax rates. Note: Thanks for Elaine Harwood for pointing some of this out to me.  (Also, see info about column 5 below) \
The **fifth column is Permno (and the 6th, 7th, and 8th are other relevant Permnos, for this company, if any)**.  I asked a Ph.D. student to match the 8-digit CUSIPs (described above) to permnos, then to fill in missing Permnos by checking 6-digit CNUMS (as described above), and then to do a hand search to fill in any additional missing Permnos.  However, there are still missing values.  This feature is fairly new, so if you decide to merge by Permno, you might want to verify the accuracy of some of your matches by hand and/or also use Cusip/Cnum to match.

4.1.3 CUSIP/CNUM/CIC
CUSIP is a nine-character alphanumeric identifier assigned to individual financial assets by an independent agency. \
As a rule, the first six characters of the CUSIP can be identified with a company; the next two characters (the seventh and eighth) identify a particular asset (e.g., a class of stock or a bond issue) issued by the company; and the ninth digit is a "check digit" to improve the accuracy of electronic transmission of CUSIPs.
**Within Compustat, the first six characters of the CUSIP are referred to as the CNUM (CUSIP issuer code)**, \
and the **last three characters of the CUSIP are called the CIC (CUSIP issue code)**. Throughout much of the history of Compustat data files, the CNUM was used as a company identifier, but it has recently been supplanted in this function by the GVKEY. In any event, the Compustat files do not track a company's CUSIP history. Rather, only the most recent CUSIP or CNUM is included.

# Raw_df

In [78]:
raw_df = pd.read_parquet('Generated Data/raw_df.parquet')
len_0 = raw_df['gvkey'].nunique()
print(raw_df['gvkey'].nunique(),'firms')
print(raw_df.shape)
# identifier to match MTR
cusip_raw = set(raw_df['cusip'].unique())
# cusip_raw = set([cusip[:-1] for cusip in cusip_raw])
raw_df['cusip_8'] =  raw_df['cusip'].apply(lambda x: x[:-1])
gvkey_raw = set(raw_df['gvkey'].unique())
tkr_raw = set(raw_df['tic'].unique())
cusip_raw = set(raw_df['cusip'].unique())
cusip8_raw = set(raw_df['cusip_8'].unique())

8024 firms
(134810, 38)


In [79]:
raw_df[raw_df.duplicated()]

,gvkey,fyear,tic,cusip,act,at,capx,ceq,ch,che,...,xrd,xrdp,xsga,cik,prcc_c,mkvalt,prcc_f,sic,age,cusip_8


In [23]:
len(raw_df[raw_df['cusip_8'] == '00036110'])

47

In [80]:
MTR_df = pd.read_csv('Generated Data/MTR_duke.csv')
MTR_compustat_df = pd.read_csv('Generated Data/MTR_compustat.csv')

# Duke data is strictly better
print('Duke:', MTR_df['CODE'].nunique(), 'len:', len(MTR_df),min(MTR_df['fyear']), max(MTR_df['fyear']))
print('Duke:', MTR_compustat_df['gvkey'].nunique(), 'len:', len(MTR_compustat_df),
      min(MTR_compustat_df['year']), max(MTR_compustat_df['year']))
# Clean MTR Data
MTR_df = MTR_df.drop_duplicates(keep='last')
MTR_df = MTR_df.replace('.', None)
MTR_df['gvkey'].replace(-999, None, inplace=True)
# Firm Identifiers
CODE_MTR = set(MTR_df['CODE'].unique()) ## equivalent to first 8 digits of CUSIP
gvkey_MTR = set(MTR_df['gvkey'].unique())
permno_MTR = set(MTR_df['Permno'].unique())

link_df = pd.read_csv('Generated Data/crsp_compustat_link_permno.csv')
link_df['LPERMNO'] = link_df['LPERMNO'].astype(str)
link_df['LPERMCO'] = link_df['LPERMCO'].astype(str)
link_df['cusip_8'] =  link_df['cusip'].apply(lambda x: x[:-1])
gvkey_link = set(link_df['gvkey'].unique())
lpermno_link = set(link_df['LPERMNO'].unique())
tic_link = set(link_df['tic'].unique())
lpermco_link = set(link_df['LPERMCO'].unique())
cusip_link = set(link_df['cusip'].unique())
cusip8_link = set(link_df['cusip_8'].unique())

# Check duplicates
MTR_df[MTR_df.duplicated()]
MTR_compustat_df[MTR_compustat_df.duplicated()]

Duke: 33191 len: 247267 1980 2021
Duke: 19428 len: 198772 1980 2016


,gvkey,year,bcg_mtrnoint,bcg_mtrint,conm,tic,cusip,cik,sic,naics


In [25]:
# limit data to regression period
period_bool = MTR_df['fyear'].apply(lambda x: True if (x>=1975) and (x<=2021) else False) # 2009
MTR_df = MTR_df[period_bool]
# Check
# MTR_df[MTR_df['MTR_BI'] == '.']
# MTR_df[MTR_df['MTR_AI'] == '.']
MTR_df.info()
print(list(MTR_df['fyear'].unique()))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 247267 entries, 0 to 247266
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   CODE    247267 non-null  object
 1   fyear   247267 non-null  int64 
 2   MTR_BI  205329 non-null  object
 3   MTR_AI  242908 non-null  object
 4   Permno  154036 non-null  object
 5   gvkey   171256 non-null  object
dtypes: int64(1), object(5)
memory usage: 13.2+ MB
[1987, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 1985, 1988, 2014, 2015, 2016, 2017, 2018, 1984, 1986, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2019, 2020, 2021, 1980, 1981, 1982, 1983]


In [26]:
print('link_df firms', len(cusip8_link))
print('raw_df firms ', len(cusip8_raw))
print('MTR_df firms ', len(CODE_MTR))
print('link_df gvkey has more match on raw_df than cusip:',len(gvkey_link & gvkey_raw),
      len(cusip8_link & cusip8_raw),len(cusip_link & cusip_raw), 
      '--> use gvkey to match link_df and raw_df')
print('link_df match on code with MTR:', len(cusip8_link & CODE_MTR))
print('raw_df match on code with MTR:', len(cusip8_raw & CODE_MTR))

link_df firms 27243
raw_df firms  8024
MTR_df firms  33191
link_df gvkey has more match on raw_df than cusip: 7470 7467 7467 --> use gvkey to match link_df and raw_df
link_df match on code with MTR: 19596
raw_df match on code with MTR: 6305


In [27]:
permno_intersection = list(lpermno_link & permno_MTR)
link_df['match_permno'] = link_df['LPERMNO'].isin(permno_intersection)
len(permno_intersection), link_df.query('match_permno == True')['gvkey'].nunique()

(15108, 15141)

In [28]:
# testing matching method results
## linked_df gvkey intersection with raw_df and Permno matching MTR_df, 
## check if the results from permno matching method used on link_df with MTR_df matches code-matching method
## using cusip_8 of raw_df and MTR_df
gvkey_matching = gvkey_raw & gvkey_MTR

raw_matched_gvkey = list(gvkey_raw & gvkey_link)
link_df['raw_matched_gvkey'] = link_df['gvkey'].isin(raw_matched_gvkey)
x = list(link_df.query('raw_matched_gvkey == True & match_permno == True')['gvkey'].unique())
# x is complete subset of gvkey matched using code-matching method, they yield the same result
print('permno matching with Raw_df and MTR_df(those that can be matched with Raw_df on gvkey):',len(x), 
      '\ndirect gvkey matching with Raw_df and MTR_df:', len(gvkey_matching),
      '\nx is not complete subset', len(set(x) & set(gvkey_matching)), len(set(x) - set(gvkey_matching)))
### permno matching mostly falls in with code matching, 501 firms are not in Raw_df

permno matching with Raw_df and MTR_df(those that can be matched with Raw_df on gvkey): 4899 
direct gvkey matching with Raw_df and MTR_df: 5807 
x is not complete subset 4515 384


In [29]:
### Matching Raw_df with MTR_df
# 1. using CODE = match on first 8 digits of cusip
intersection_code = set(raw_df['cusip_8']) & CODE_MTR
unmatched_code = set(raw_df['cusip_8']) - CODE_MTR
print(len_0, 'initial matching loss:', round(len(unmatched_code)/len_0 * 100),'%')
print(f'1. Matching Raw_df and MTR_df on cusip_8: matched: {len(intersection_code)}, loss: {len(unmatched_code)}')
#raw_df['match_MTR'] = raw_df['cusip_8'].progress_apply(lambda x: True if x in intersection_code else False)
raw_df['match_MTR'] = raw_df['cusip_8'].isin(intersection_code)

## Those in matched_MTR == True
#raw_df[raw_df['match_MTR'] == True]['gvkey'].nunique()
gvkey_matched = list(raw_df.query('match_MTR == True')['gvkey'].unique())
print('gvkey of cusip_8 matched',len(gvkey_matched))
link_df['raw_gvkey_matched'] = link_df['gvkey'].isin(gvkey_matched)
print('raw_df <-cusip_8-> link_df <-Permno-> MTR_df:',link_df.query('raw_gvkey_matched == True & match_permno == True')['gvkey'].nunique())
print('raw_df <-cusip_8-> link_df <-XPermnoX-> MTR_df:',link_df.query('raw_gvkey_matched == True & match_permno == False')['gvkey'].nunique())
print(link_df.query('raw_gvkey_matched == True')['gvkey'].nunique())
## Those in matched_MTR == False
gvkey_unmatched = list(raw_df.query('match_MTR == False')['gvkey'].unique())
#list(raw_df[~raw_df['cusip_8'].isin(intersection_code)]['gvkey'].unique())

print('gvkey of cusip_8 unmatched',len(gvkey_unmatched))
# unmatched gvkey linked with link_df gvkey
raw_unmatched_gvkey = list(set(gvkey_unmatched) & gvkey_link)
link_df['raw_unmatched_gvkey'] = link_df['gvkey'].isin(raw_unmatched_gvkey)

# 1.1 find if those in link_df with unmatched raw gvkey has matchings with MTR_df on Permno
loss_recover_permno = list(link_df.query('raw_unmatched_gvkey == True & match_permno == True')['gvkey'].unique())
print('1.1. recovery of matching loss with link_df and MTR_df matching on permno:', len(loss_recover_permno))
raw_df['loss_recover_permno'] = raw_df['gvkey'].isin(loss_recover_permno)
total_match = raw_df.query('match_MTR == True | loss_recover_permno == True')['gvkey'].nunique()
print(f'Total matched so far: {len(gvkey_matched) + len(loss_recover_permno)} {total_match}')
#print(f'(intersection of cusip_8 matched and unmatched(0=no gvkey overlap): {len(set(gvkey_unmatched) & set(gvkey_matched))})')

# 1.2 find if those in Raw_df still in loss could be matched with MTR_df using gvkey
gvkey_unmatched = list(raw_df.query('match_MTR == False & loss_recover_permno == False')['gvkey'].unique())
loss_recover_gvkey = set(gvkey_unmatched) & gvkey_MTR
print('1.2 recovery of matching loss with raw_df and MTR_df matching on gvkey:',len(gvkey_MTR & set(loss_recover_gvkey)))

raw_df['loss_recover_gvkey'] = raw_df['gvkey'].isin(loss_recover_gvkey)
print(f'matched using code and gvkey = {len(gvkey_matched) + len(loss_recover_permno) + len(loss_recover_gvkey)},',
      raw_df.query('match_MTR == True | loss_recover_permno == True | loss_recover_gvkey == True')['gvkey'].nunique())
total_match = raw_df.query('match_MTR == True | loss_recover_permno == True | loss_recover_gvkey == True')['gvkey'].nunique()
print(f'Total matched so far: {len(gvkey_matched) + len(loss_recover_permno) + len(loss_recover_gvkey)} {total_match}')

gvkey_matching = len(gvkey_raw & gvkey_MTR)

matching_loss = raw_df.query('match_MTR == False & loss_recover_permno == False & loss_recover_gvkey == False')['gvkey'].nunique()
print(f'matching loss {matching_loss}, {round(matching_loss/len_0 * 100)}%')
print('minimized loss relative to simple gvkey matching between Raw_df and MTR_df:',(len_0 - gvkey_matching) - (len_0 - total_match))

8024 initial matching loss: 21 %
1. Matching Raw_df and MTR_df on cusip_8: matched: 6305, loss: 1719
gvkey of cusip_8 matched 6305
raw_df <-cusip_8-> link_df <-Permno-> MTR_df: 4799
raw_df <-cusip_8-> link_df <-XPermnoX-> MTR_df: 1774
6280
gvkey of cusip_8 unmatched 1719
1.1. recovery of matching loss with link_df and MTR_df matching on permno: 100
Total matched so far: 6405 6405
1.2 recovery of matching loss with raw_df and MTR_df matching on gvkey: 31
matched using code and gvkey = 6436, 6436
Total matched so far: 6436 6436
matching loss 1588, 20%
minimized loss relative to simple gvkey matching between Raw_df and MTR_df: 629


In [30]:
print('raw_df firms:',len(gvkey_raw))
print('MTR_df firms:',len(CODE_MTR))

permno_intersection = list(lpermno_link & permno_MTR)
link_df['match_permno'] = link_df['LPERMNO'].isin(permno_intersection)
print('link_df <-Permno-> MTR_df:', link_df.query('match_permno == True')['gvkey'].nunique())

intersection_code = set(raw_df['cusip_8']) & CODE_MTR
unmatched_code = set(raw_df['cusip_8']) - CODE_MTR
raw_df['match_MTR'] = raw_df['cusip_8'].isin(intersection_code)
print('<Match 1>raw_df <-cusip_8-> MTR_df:', raw_df.query('match_MTR == True')['gvkey'].nunique())

# gvkey of unmatched cusip_8 with MTR_df
gvkey_unmatched = list(raw_df.query('match_MTR == False')['gvkey'].unique())
# 'unmatched with MTR_df' matched with link_df
raw_unmatched_gvkey = list(set(gvkey_unmatched) & gvkey_link)
link_df['raw_unmatched_gvkey'] = link_df['gvkey'].isin(raw_unmatched_gvkey)
print('unmatched raw_df <-gvkey-> link_df:', link_df.query('raw_unmatched_gvkey == True')['gvkey'].nunique())

loss_recover_permno = list(link_df.query('raw_unmatched_gvkey == True & match_permno == True')['gvkey'].unique())
raw_df['loss_recover_permno'] = raw_df['gvkey'].isin(loss_recover_permno)
print('<Match 2>unmatched raw_df <-gvkey-> link_df with <-Permno-> MTR_df:', raw_df.query('loss_recover_permno == True')['gvkey'].nunique())

# gvkey of raw_df unmatched with MTR_df by cusip_8 and Permno
gvkey_unmatched = list(raw_df.query('match_MTR == False & loss_recover_permno == False')['gvkey'].unique())
# directly match on gvkey
loss_recover_gvkey = set(gvkey_unmatched) & gvkey_MTR
raw_df['loss_recover_gvkey'] = raw_df['gvkey'].isin(loss_recover_gvkey)
print('<Match 3>unmatched raw_df <-gvkey-> MTR_df:', raw_df.query('loss_recover_gvkey == True')['gvkey'].nunique())

print('total match:',raw_df.query('match_MTR == True | loss_recover_permno == True | loss_recover_gvkey == True')['gvkey'].nunique())

raw_df firms: 8024
MTR_df firms: 33191
link_df <-Permno-> MTR_df: 15141
<Match 1>raw_df <-cusip_8-> MTR_df: 6305
unmatched raw_df <-gvkey-> link_df: 1190
<Match 2>unmatched raw_df <-gvkey-> link_df with <-Permno-> MTR_df: 100
<Match 3>unmatched raw_df <-gvkey-> MTR_df: 31
total match: 6436


In [31]:
matched_raw = list(raw_df.query('match_MTR == True | loss_recover_permno == True | loss_recover_gvkey == True')['gvkey'].unique())

In [32]:
MTR_df = MTR_df.set_index('fyear')

In [33]:
new_MTR = MTR_df.copy()[0:0]
intersection_code = set(raw_df['cusip_8']) & CODE_MTR
loss_recover_permno = list(raw_df.query('loss_recover_permno == True')['cusip_8'])
loss_recover_gvkey = list(raw_df.query('loss_recover_gvkey == True')['cusip_8'])

for k in tqdm(set(raw_df['cusip_8'])):
    mini_df = new_MTR[0:0]
    # 1. Match on cusip_8 and CODE
    if k in intersection_code:
        #print('Match on cusip_8 and CODE')
        mini_df = MTR_df.loc[MTR_df['CODE'] == k]
    # 2. Match rest on permno
    elif k in loss_recover_permno:
        #print('Match loss_recover_permno')
        mini_df = MTR_df.loc[MTR_df['CODE'] == k]
    # 3. Match rest on gvkey
    elif k in loss_recover_gvkey:
        #print('Match loss_recover_gvkey')
        mini_df = MTR_df.loc[MTR_df['CODE'] == k]
    else:
        # 4. Cannot be matched, do not include in MTR_df
        continue
    
    key_MTR_years = set(MTR_df.loc[MTR_df['CODE'] == k].index)
    key_raw_period = set(raw_df.loc[raw_df['cusip_8'] == k]['fyear'])
    # years missing in MTR but exist in raw period for each firms
    y_missing = list(set(key_raw_period) - set(key_MTR_years))
    for y in y_missing:
        # create empty MTR data with raw period missing in MTR years
        mini_df.loc[y,:] = None
        mini_df.loc[y,'CODE'] = k
        mini_df.loc[y,'gvkey'] = raw_df.query('cusip_8 == @k')['gvkey'].unique()[0] #raw_df.loc[raw_df['cusip_8'] == k]['gvkey'].unique()[0]
    
    mini_df = mini_df.sort_index(ascending=True)
    new_MTR = pd.concat([new_MTR, mini_df], axis=0)
    #print(new_MTR)

MTR_df = new_MTR
MTR_df = MTR_df.reset_index()

100%|███████████████████████████████████████| 8024/8024 [06:34<00:00, 20.35it/s]


In [34]:
len(set(raw_df['cusip_8']))

8024

In [35]:
print(set(intersection_code) & set(loss_recover_permno))
print(set(intersection_code) & set(loss_recover_gvkey))
print(set(loss_recover_permno) & set(loss_recover_gvkey))

set()
set()
set()


In [36]:
MTR_df[MTR_df.duplicated()]
#MTR_df.groupby('CODE')['fyear'].count()

,fyear,CODE,MTR_BI,MTR_AI,Permno,gvkey


In [37]:
MTR_df['CODE'].nunique()
#MTR_df[MTR_df['gvkey'].isnull()] - some gvkeys are missing cuz most matching was done on cusip_8 and CODE

6436

In [38]:
MTR_df['CODE'].nunique(), raw_df['cusip_8'].nunique()

(6436, 8024)

In [39]:
MTR_df['gvkey'].nunique(), raw_df['gvkey'].nunique()

(6436, 8024)

In [40]:
raw_df.query('match_MTR == True | loss_recover_permno == True | loss_recover_gvkey == True')['gvkey'].nunique()

6436

In [41]:
# MTR_keys = set(MTR_df[~MTR_df['gvkey'].isnull()]['gvkey'].unique())
# raw_keys = set(raw_df['gvkey'].unique())
# intersection = list(MTR_keys & raw_keys)
# len(intersection)
MTR_code = set(MTR_df['CODE'].unique())
raw_cusip_8 = set(raw_df['cusip_8'].unique())
intersection = list(MTR_code & raw_cusip_8)
print(len(intersection))

print('lost:',raw_df.query('match_MTR == False & loss_recover_permno == False & loss_recover_gvkey == False')['gvkey'].nunique())
raw_df['in_MTR'] = raw_df['cusip_8'].isin(intersection)
print(len(intersection) + raw_df.query('match_MTR == False & loss_recover_permno == False & loss_recover_gvkey == False')['cusip_8'].nunique())
raw_df.query('in_MTR == True')['cusip_8'].nunique()

6436
lost: 1588
8024


6436

In [42]:
1933 / 9180 * 100

21.05664488017429

In [43]:
1929 / 9147 * 100

21.08888160052476

In [44]:
1588 / 8024 * 100

19.790628115653043

In [45]:
### Match MTR_df and raw_df
raw_df = raw_df.rename(columns={'cusip_8':'CODE'})
rawMTR_df = pd.merge(raw_df, MTR_df[['CODE','fyear','MTR_BI','MTR_AI']], on=['CODE', 'fyear'])
rawMTR_df['MTR_BI'] = rawMTR_df['MTR_BI'].apply(lambda x: None if x == None else float(x))
rawMTR_df['MTR_AI'] = rawMTR_df['MTR_AI'].apply(lambda x: None if x == None else float(x))
rawMTR_df['gvkey'].nunique()

6436

In [46]:
### handling missing MTR with mean of the sample
MTR_BI_mean = rawMTR_df['MTR_BI'].mean()
MTR_AI_mean = rawMTR_df['MTR_AI'].mean()
print(MTR_BI_mean, MTR_AI_mean)

rawMTR_df.loc[rawMTR_df['MTR_BI'].isnull() == True, 'MTR_BI'] = MTR_BI_mean
rawMTR_df.loc[rawMTR_df['MTR_AI'].isnull() == True, 'MTR_AI'] = MTR_AI_mean

0.30573803977529346 0.2089795148954314


In [47]:
print(rawMTR_df['gvkey'].nunique(), rawMTR_df['CODE'].nunique())
rawMTR_df = rawMTR_df.drop(columns = ['CODE','match_MTR','loss_recover_permno','loss_recover_gvkey','in_MTR'])

6436 6436


In [48]:
rawMTR_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 126936 entries, 0 to 126935
Data columns (total 40 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   gvkey   126936 non-null  int64  
 1   fyear   126936 non-null  float64
 2   tic     126936 non-null  object 
 3   cusip   126936 non-null  object 
 4   act     124163 non-null  float64
 5   at      126936 non-null  float64
 6   capx    125803 non-null  float64
 7   ceq     126717 non-null  float64
 8   ch      118340 non-null  float64
 9   che     126887 non-null  float64
 10  cogs    126926 non-null  float64
 11  csho    125850 non-null  float64
 12  dd1     124264 non-null  float64
 13  dlc     126803 non-null  float64
 14  dltt    126605 non-null  float64
 15  dt      61982 non-null   float64
 16  ebit    125328 non-null  float64
 17  ebitda  126679 non-null  float64
 18  gdwl    85044 non-null   float64
 19  intan   115965 non-null  float64
 20  intano  53088 non-null   float64
 21  lt      12

In [49]:
# save temporarily
rawMTR_df.to_parquet('Generated Data/rawMTR_df.parquet')
rawMTR_df['gvkey'].nunique()

6436

# Reg_df

In [50]:
reg_df = pd.read_parquet('Generated Data/reg_df.parquet') # atsale_only
len_0 = reg_df['gvkey'].nunique()
print(reg_df['gvkey'].nunique(),'firms')

# identifier to match MTR
cusip_reg = set(reg_df['cusip'].unique())
# cusip_reg = set([cusip[:-1] for cusip in cusip_reg])
reg_df['cusip_8'] =  reg_df['cusip'].apply(lambda x: x[:-1])
gvkey_reg = set(reg_df['gvkey'].unique())
tkr_reg = set(reg_df['tic'].unique())
cusip_reg = set(reg_df['cusip'].unique())
cusip8_reg = set(reg_df['cusip_8'].unique())

4247 firms


In [51]:
MTR_df = pd.read_csv('Generated Data/MTR_duke.csv')
MTR_compustat_df = pd.read_csv('Generated Data/MTR_compustat.csv')

# Duke data is strictly better
print('Duke:', MTR_df['CODE'].nunique(), 'len:', len(MTR_df),min(MTR_df['fyear']), max(MTR_df['fyear']))
print('Duke:', MTR_compustat_df['gvkey'].nunique(), 'len:', len(MTR_compustat_df),
      min(MTR_compustat_df['year']), max(MTR_compustat_df['year']))
# Clean MTR Data
MTR_df = MTR_df.drop_duplicates(keep='last')
MTR_df = MTR_df.replace('.', None)
MTR_df['gvkey'].replace(-999, None, inplace=True)
# Firm Identifiers
CODE_MTR = set(MTR_df['CODE'].unique()) ## equivalent to first 8 digits of CUSIP
gvkey_MTR = set(MTR_df['gvkey'].unique())
permno_MTR = set(MTR_df['Permno'].unique())

link_df = pd.read_csv('Generated Data/crsp_compustat_link_permno.csv')
link_df['LPERMNO'] = link_df['LPERMNO'].astype(str)
link_df['LPERMCO'] = link_df['LPERMCO'].astype(str)
link_df['cusip_8'] =  link_df['cusip'].apply(lambda x: x[:-1])
gvkey_link = set(link_df['gvkey'].unique())
lpermno_link = set(link_df['LPERMNO'].unique())
tic_link = set(link_df['tic'].unique())
lpermco_link = set(link_df['LPERMCO'].unique())
cusip_link = set(link_df['cusip'].unique())
cusip8_link = set(link_df['cusip_8'].unique())

# limit data to regression period
period_bool = MTR_df['fyear'].apply(lambda x: True if (x>=2009) and (x<=2021) else False)
MTR_df = MTR_df[period_bool]
# Check
# MTR_df[MTR_df['MTR_BI'] == '.']
# MTR_df[MTR_df['MTR_AI'] == '.']
MTR_df.info()
print(list(MTR_df['fyear'].unique()))

Duke: 33191 len: 247267 1980 2021
Duke: 19428 len: 198772 1980 2016
<class 'pandas.core.frame.DataFrame'>
Int64Index: 65305 entries, 18 to 247266
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   CODE    65305 non-null  object
 1   fyear   65305 non-null  int64 
 2   MTR_BI  48772 non-null  object
 3   MTR_AI  63837 non-null  object
 4   Permno  22319 non-null  object
 5   gvkey   54197 non-null  object
dtypes: int64(1), object(5)
memory usage: 3.5+ MB
[2014, 2015, 2016, 2017, 2018, 2009, 2010, 2011, 2012, 2013, 2019, 2020, 2021]


In [52]:
print('link_df firms', len(cusip8_link))
print('reg_df firms ', len(cusip8_reg))
print('MTR_df firms ', len(CODE_MTR))
print('link_df gvkey has more match on reg_df than cusip:',len(gvkey_link & gvkey_reg),
      len(cusip8_link & cusip8_reg),len(cusip_link & cusip_reg), 
      '--> use gvkey to match link_df and reg_df')
print('link_df match on code with MTR:', len(cusip8_link & CODE_MTR))
print('reg_df match on code with MTR:', len(cusip8_reg & CODE_MTR))

link_df firms 27243
reg_df firms  4247
MTR_df firms  33191
link_df gvkey has more match on reg_df than cusip: 3982 3979 3979 --> use gvkey to match link_df and reg_df
link_df match on code with MTR: 19596
reg_df match on code with MTR: 3339


In [53]:
permno_intersection = list(lpermno_link & permno_MTR)
link_df['match_permno'] = link_df['LPERMNO'].isin(permno_intersection)
len(permno_intersection), link_df.query('match_permno == True')['gvkey'].nunique()

(15108, 15141)

In [54]:
### Matching reg_df with MTR_df
# 1. using CODE = match on first 8 digits of cusip
intersection_code = set(reg_df['cusip_8']) & CODE_MTR
unmatched_code = set(reg_df['cusip_8']) - CODE_MTR
print(len_0, 'initial matching loss:', round(len(unmatched_code)/len_0 * 100),'%')
print(f'1. Matching reg_df and MTR_df on cusip_8: matched: {len(intersection_code)}, loss: {len(unmatched_code)}')
#reg_df['match_MTR'] = reg_df['cusip_8'].progress_apply(lambda x: True if x in intersection_code else False)
reg_df['match_MTR'] = reg_df['cusip_8'].isin(intersection_code)

## Those in matched_MTR == True
#reg_df[reg_df['match_MTR'] == True]['gvkey'].nunique()
gvkey_matched = list(reg_df.query('match_MTR == True')['gvkey'].unique())
print('gvkey of cusip_8 matched',len(gvkey_matched))
link_df['reg_gvkey_matched'] = link_df['gvkey'].isin(gvkey_matched)
print('reg_df <-cusip_8-> link_df <-Permno-> MTR_df:',link_df.query('reg_gvkey_matched == True & match_permno == True')['gvkey'].nunique())
print('reg_df <-cusip_8-> link_df <-XPermnoX-> MTR_df:',link_df.query('reg_gvkey_matched == True & match_permno == False')['gvkey'].nunique())
print(link_df.query('reg_gvkey_matched == True')['gvkey'].nunique())
## Those in matched_MTR == False
gvkey_unmatched = list(reg_df.query('match_MTR == False')['gvkey'].unique())
#list(reg_df[~reg_df['cusip_8'].isin(intersection_code)]['gvkey'].unique())

print('gvkey of cusip_8 unmatched',len(gvkey_unmatched))
# unmatched gvkey linked with link_df gvkey
reg_unmatched_gvkey = list(set(gvkey_unmatched) & gvkey_link)
link_df['reg_unmatched_gvkey'] = link_df['gvkey'].isin(reg_unmatched_gvkey)

# 1.1 find if those in link_df with unmatched reg gvkey has matchings with MTR_df on Permno
loss_recover_permno = list(link_df.query('reg_unmatched_gvkey == True & match_permno == True')['gvkey'].unique())
print('1.1. recovery of matching loss with link_df and MTR_df matching on permno:', len(loss_recover_permno))
reg_df['loss_recover_permno'] = reg_df['gvkey'].isin(loss_recover_permno)
total_match = reg_df.query('match_MTR == True | loss_recover_permno == True')['gvkey'].nunique()
print(f'Total matched so far: {len(gvkey_matched) + len(loss_recover_permno)} {total_match}')
#print(f'(intersection of cusip_8 matched and unmatched(0=no gvkey overlap): {len(set(gvkey_unmatched) & set(gvkey_matched))})')

# 1.2 find if those in reg_df still in loss could be matched with MTR_df using gvkey
gvkey_unmatched = list(reg_df.query('match_MTR == False & loss_recover_permno == False')['gvkey'].unique())
loss_recover_gvkey = set(gvkey_unmatched) & gvkey_MTR
print('1.2 recovery of matching loss with reg_df and MTR_df matching on gvkey:',len(gvkey_MTR & set(loss_recover_gvkey)))

reg_df['loss_recover_gvkey'] = reg_df['gvkey'].isin(loss_recover_gvkey)
print(f'matched using code and gvkey = {len(gvkey_matched) + len(loss_recover_permno) + len(loss_recover_gvkey)},',
      reg_df.query('match_MTR == True | loss_recover_permno == True | loss_recover_gvkey == True')['gvkey'].nunique())
total_match = reg_df.query('match_MTR == True | loss_recover_permno == True | loss_recover_gvkey == True')['gvkey'].nunique()
print(f'Total matched so far: {len(gvkey_matched) + len(loss_recover_permno) + len(loss_recover_gvkey)} {total_match}')

gvkey_matching = len(gvkey_reg & gvkey_MTR)

matching_loss = reg_df.query('match_MTR == False & loss_recover_permno == False & loss_recover_gvkey == False')['gvkey'].nunique()
print(f'matching loss {matching_loss}, {round(matching_loss/len_0 * 100)}%')
print('minimized loss relative to simple gvkey matching between reg_df and MTR_df:',(len_0 - gvkey_matching) - (len_0 - total_match))

4247 initial matching loss: 21 %
1. Matching reg_df and MTR_df on cusip_8: matched: 3339, loss: 908
gvkey of cusip_8 matched 3339
reg_df <-cusip_8-> link_df <-Permno-> MTR_df: 1939
reg_df <-cusip_8-> link_df <-XPermnoX-> MTR_df: 1564
3331
gvkey of cusip_8 unmatched 908
1.1. recovery of matching loss with link_df and MTR_df matching on permno: 65
Total matched so far: 3404 3404
1.2 recovery of matching loss with reg_df and MTR_df matching on gvkey: 31
matched using code and gvkey = 3435, 3435
Total matched so far: 3435 3435
matching loss 812, 19%
minimized loss relative to simple gvkey matching between reg_df and MTR_df: 365


In [55]:
print('reg_df firms:',len(gvkey_reg))
print('MTR_df firms:',len(CODE_MTR))

permno_intersection = list(lpermno_link & permno_MTR)
link_df['match_permno'] = link_df['LPERMNO'].isin(permno_intersection)
print('link_df <-Permno-> MTR_df:', link_df.query('match_permno == True')['gvkey'].nunique())

intersection_code = set(reg_df['cusip_8']) & CODE_MTR
unmatched_code = set(reg_df['cusip_8']) - CODE_MTR
reg_df['match_MTR'] = reg_df['cusip_8'].isin(intersection_code)
print('<Match 1>reg_df <-cusip_8-> MTR_df:', reg_df.query('match_MTR == True')['gvkey'].nunique())

# gvkey of unmatched cusip_8 with MTR_df
gvkey_unmatched = list(reg_df.query('match_MTR == False')['gvkey'].unique())
# 'unmatched with MTR_df' matched with link_df
reg_unmatched_gvkey = list(set(gvkey_unmatched) & gvkey_link)
link_df['reg_unmatched_gvkey'] = link_df['gvkey'].isin(reg_unmatched_gvkey)
print('unmatched reg_df <-gvkey-> link_df:', link_df.query('reg_unmatched_gvkey == True')['gvkey'].nunique())

loss_recover_permno = list(link_df.query('reg_unmatched_gvkey == True & match_permno == True')['gvkey'].unique())
reg_df['loss_recover_permno'] = reg_df['gvkey'].isin(loss_recover_permno)
print('<Match 2>unmatched reg_df <-gvkey-> link_df with <-Permno-> MTR_df:', reg_df.query('loss_recover_permno == True')['gvkey'].nunique())

# gvkey of reg_df unmatched with MTR_df by cusip_8 and Permno
gvkey_unmatched = list(reg_df.query('match_MTR == False & loss_recover_permno == False')['gvkey'].unique())
# directly match on gvkey
loss_recover_gvkey = set(gvkey_unmatched) & gvkey_MTR
reg_df['loss_recover_gvkey'] = reg_df['gvkey'].isin(loss_recover_gvkey)
print('<Match 3>unmatched reg_df <-gvkey-> MTR_df:', reg_df.query('loss_recover_gvkey == True')['gvkey'].nunique())

print('total match:',reg_df.query('match_MTR == True | loss_recover_permno == True | loss_recover_gvkey == True')['gvkey'].nunique())

reg_df firms: 4247
MTR_df firms: 33191
link_df <-Permno-> MTR_df: 15141
<Match 1>reg_df <-cusip_8-> MTR_df: 3339
unmatched reg_df <-gvkey-> link_df: 651
<Match 2>unmatched reg_df <-gvkey-> link_df with <-Permno-> MTR_df: 65
<Match 3>unmatched reg_df <-gvkey-> MTR_df: 31
total match: 3435


In [56]:
matched_reg = list(reg_df.query('match_MTR == True | loss_recover_permno == True | loss_recover_gvkey == True')['gvkey'].unique())

In [57]:
print('MTR matched raw firms:', len(matched_raw))
print('MTR matched reg firms:', len(matched_reg))
print('intersection', len(set(matched_raw) & set(matched_reg)))
print('matched_reg is complete subset of matched_raw:', len(set(matched_reg) - set(matched_raw)), len(set(matched_raw) - set(matched_reg)))

MTR matched raw firms: 6436
MTR matched reg firms: 3435
intersection 3435
matched_reg is complete subset of matched_raw: 0 3001


In [58]:
MTR_df = MTR_df.set_index('fyear')

In [59]:
new_MTR = MTR_df.copy()[0:0]
intersection_code = set(reg_df['cusip_8']) & CODE_MTR
loss_recover_permno = list(reg_df.query('loss_recover_permno == True')['cusip_8'])
loss_recover_gvkey = list(reg_df.query('loss_recover_gvkey == True')['cusip_8'])

for k in tqdm(set(reg_df['cusip_8'])):
    mini_df = mini_df[0:0]
    # 1. Match on cusip_8 and CODE
    if k in intersection_code:
        #print('Match on cusip_8 and CODE')
        mini_df = MTR_df.loc[MTR_df['CODE'] == k]
    # 2. Match rest on permno
    elif k in loss_recover_permno:
        #print('Match loss_recover_permno')
        mini_df = MTR_df.loc[MTR_df['CODE'] == k]
    # 3. Match rest on gvkey
    elif k in loss_recover_gvkey:
        #print('Match loss_recover_gvkey')
        mini_df = MTR_df.loc[MTR_df['CODE'] == k]
    else:
        # 4. Cannot be matched, do not include in MTR_df
        continue
    
    key_MTR_years = set(MTR_df.loc[MTR_df['CODE'] == k].index)
    key_reg_period = set(reg_df.loc[reg_df['cusip_8'] == k]['fyear'])
    # years missing in MTR but exist in reg period for each firms
    y_missing = list(set(key_reg_period) - set(key_MTR_years))
    for y in y_missing:
        # create empty MTR data with reg period missing in MTR years
        mini_df.loc[y,:] = None
        mini_df.loc[y,'CODE'] = k
        mini_df.loc[y,'gvkey'] = reg_df.query('cusip_8 == @k')['gvkey'].unique()[0] #reg_df.loc[reg_df['cusip_8'] == k]['gvkey'].unique()[0]
    
    mini_df = mini_df.sort_index(ascending=True)
    new_MTR = pd.concat([new_MTR, mini_df], axis=0)
    #print(new_MTR)

MTR_df = new_MTR
MTR_df = MTR_df.reset_index()

100%|███████████████████████████████████████| 4247/4247 [00:52<00:00, 80.80it/s]


In [60]:
len(set(reg_df['cusip_8']))

4247

In [61]:
print(set(intersection_code) & set(loss_recover_permno))
print(set(intersection_code) & set(loss_recover_gvkey))
print(set(loss_recover_permno) & set(loss_recover_gvkey))

set()
set()
set()


In [62]:
MTR_df[MTR_df.duplicated()]

,fyear,CODE,MTR_BI,MTR_AI,Permno,gvkey


In [63]:
print(MTR_df['CODE'].nunique())
print(MTR_df['CODE'].nunique(), reg_df['cusip_8'].nunique())
print(MTR_df['gvkey'].nunique(), reg_df['gvkey'].nunique())
reg_df.query('match_MTR == True | loss_recover_permno == True | loss_recover_gvkey == True')['gvkey'].nunique()

3435
3435 4247
3435 4247


3435

In [64]:
# MTR_keys = set(MTR_df[~MTR_df['gvkey'].isnull()]['gvkey'].unique())
# reg_keys = set(reg_df['gvkey'].unique())
# intersection = list(MTR_keys & reg_keys)
# len(intersection)
MTR_code = set(MTR_df['CODE'].unique())
reg_cusip_8 = set(reg_df['cusip_8'].unique())
intersection = list(MTR_code & reg_cusip_8)
print(len(intersection))

print('lost:',reg_df.query('match_MTR == False & loss_recover_permno == False & loss_recover_gvkey == False')['gvkey'].nunique())
reg_df['in_MTR'] = reg_df['cusip_8'].isin(intersection)
print(len(intersection) + reg_df.query('match_MTR == False & loss_recover_permno == False & loss_recover_gvkey == False')['cusip_8'].nunique())
reg_df.query('in_MTR == True')['cusip_8'].nunique()

3435
lost: 812
4247


3435

In [65]:
1099 / 5221 * 100

21.04960735491285

In [66]:
1099 / 5216 * 100

21.06978527607362

In [67]:
812 / 4247 * 100

19.119378384742173

In [68]:
868 / 4847 * 100

17.90798432019806

In [69]:
### Match MTR_df and reg_df
reg_df = reg_df.rename(columns={'cusip_8':'CODE'})
regMTR_df = pd.merge(reg_df, MTR_df[['CODE','fyear','MTR_BI','MTR_AI']], on=['CODE', 'fyear'])
regMTR_df['MTR_BI'] = regMTR_df['MTR_BI'].apply(lambda x: None if x == None else float(x))
regMTR_df['MTR_AI'] = regMTR_df['MTR_AI'].apply(lambda x: None if x == None else float(x))
regMTR_df['gvkey'].nunique()

3435

In [70]:
### handling missing MTR with mean of the sample
MTR_BI_mean = regMTR_df['MTR_BI'].mean()
MTR_AI_mean = regMTR_df['MTR_AI'].mean()
print(MTR_BI_mean, MTR_AI_mean)

regMTR_df.loc[regMTR_df['MTR_BI'].isnull() == True, 'MTR_BI'] = MTR_BI_mean
regMTR_df.loc[regMTR_df['MTR_AI'].isnull() == True, 'MTR_AI'] = MTR_AI_mean

0.23928600589927226 0.14268911568372086


In [71]:
print(regMTR_df['gvkey'].nunique(), regMTR_df['CODE'].nunique())
regMTR_df = regMTR_df.drop(columns = ['CODE','match_MTR','loss_recover_permno','loss_recover_gvkey','in_MTR'])

regMTR_df.info()

3435 3435
<class 'pandas.core.frame.DataFrame'>
Int64Index: 33015 entries, 0 to 33014
Data columns (total 40 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   gvkey   33015 non-null  int64  
 1   fyear   33015 non-null  float64
 2   tic     33015 non-null  object 
 3   cusip   33015 non-null  object 
 4   act     32389 non-null  float64
 5   at      33015 non-null  float64
 6   capx    32959 non-null  float64
 7   ceq     32925 non-null  float64
 8   ch      32612 non-null  float64
 9   che     33014 non-null  float64
 10  cogs    33015 non-null  float64
 11  csho    32649 non-null  float64
 12  dd1     32368 non-null  float64
 13  dlc     33010 non-null  float64
 14  dltt    32883 non-null  float64
 15  dt      31692 non-null  float64
 16  ebit    33014 non-null  float64
 17  ebitda  33000 non-null  float64
 18  gdwl    32556 non-null  float64
 19  intan   32595 non-null  float64
 20  intano  31908 non-null  float64
 21  lt      32929 non-null  f

In [72]:
# save temporarily
regMTR_df.to_parquet('Generated Data/regMTR_df.parquet')
regMTR_df['gvkey'].nunique()

3435

In [73]:
reg_key = set(regMTR_df['gvkey'].unique())
raw_key = set(rawMTR_df['gvkey'].unique())
# Check if reg_df is complete subset of raw_df
len(reg_key & raw_key), len(reg_key - raw_key), len(raw_key - reg_key)

(3435, 0, 3001)

# END